In this notebook, you will learn optimization techniques on the subject of Santa-2024 competition.<br>
santa-2024のコンペを題材に、最適化手法を学んでいきます。<br>
1. [Issues in Competition (コンペにおける課題)](https://www.kaggle.com/code/utm529fg/eng-tutolial-1-issues-in-competitions)
2. [Greedy (貪欲法)](https://www.kaggle.com/code/utm529fg/eng-tutolial-2-greedy)
3. BS: Beam Search (ビームサーチ) ← this notebook
4. HC: Hill Climbing (山登り法)
5. SA: Simulated Annealing (焼きなまし法)
6. GA: Genetic Algorithm (遺伝的アルゴリズム)

# 3-1. What's Beam Search Algrithm　ビームサーチとは
In the previous greedy tutorial, the idea was to decide on a sequence of 10 words, one word at a time, in order from the top to the bottom to maximize the score. In beam search, we do not settle on a single word, but keep, for example, the top three candidates and move on to the next word.<br>
The number of top candidates is called the beam width.
The algorithm is as follows.<br>
**Step 1**. Decide the `beam_width`.<br>
**Step 2**. Calculate the perplexity score for one word only.<br>
**Step 3**. The top scoring `beam_width` words are tentative text candidates.<br>
**Step 4**. Calculate the two-word perplexity score for each of the remaining candidate words as the second word.<br>
**Step 5**. The top `beam_width` words with the best scores are selected as tentative text candidates. step6.<br>
**Step 6**. Repeat steps 4-5, increasing the number of words one by one.<br>
The larger the beam_width, the more text is left as candidates, which increases the computation time, but the better the score is expected to be.<br>

前回の貪欲法では、10個の単語の並びを頭から順にスコアが最大になるように1単語ずつ決めていくという考え方でした。
ビームサーチでは、1単語に決めず、例えば上位3つまで候補を残して、次の単語の検討に移ります。<br>
上位何個まで残すかの数値を、ビーム幅と呼びます。<br>
以下のようなアルゴリズムです。<br>
**Step 1**. ビーム幅 `beam_width` を決める。<br>
**Step 2**. 1単語のみで、perplexityスコアを計算する。<br>
**Step 3**. スコアの良い上位 `beam_width` 個の単語を暫定のテキスト候補とする。<br>
**Step 4**. 候補に残りの単語をそれぞれ2番目の単語とした時の、2単語でのperplexityスコアを計算する。<br>
**Step 5**. スコアの良い上位 `beam_width` 個のテキストを暫定のテキスト候補とする。<br>
**Step 6**. 4-5の手順を単語数の数だけ一つずつ増やしながら繰り返す。<br>
ビーム幅を大きくするほど、候補として残すテキストが増えるため、計算時間は増えますが、スコアは良くなることが期待できます。

# 3-2. Implementation 実装

In [3]:
"""Evaluation metric for Santa 2024."""

import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ParticipantVisibleError(Exception):
    pass


def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
    load_in_8bit: bool = False,
    clear_mem: bool = False,
) -> float:
    """
    Calculates the mean perplexity of submitted text permutations compared to an original text.

    Parameters
    ----------
    solution : DataFrame
        DataFrame containing the original text in a column named 'text'.
        Includes a row ID column specified by `row_id_column_name`.

    submission : DataFrame
        DataFrame containing the permuted text in a column named 'text'.
        Must have the same row IDs as the solution.
        Includes a row ID column specified by `row_id_column_name`.

    row_id_column_name : str
        Name of the column containing row IDs.
        Ensures aligned comparison between solution and submission.

    model_path : str, default='/kaggle/input/gemma-2/transformers/gemma-2-9b/2'
        Path to the serialized LLM.

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    clear_mem : bool, default=False
        Clear GPU memory after scoring by clearing the CUDA cache.
        Useful for testing.

    Returns
    -------
    float
        The mean perplexity score. Lower is better.

    Raises
    ------
    ParticipantVisibleError
        If the submission format is invalid or submitted strings are not valid permutations.

    Examples
    --------
    >>> import pandas as pd
    >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
    >>> solution = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["this is a normal english sentence", "the quick brown fox jumps over the lazy dog"]
    ... })
    >>> submission = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["sentence english normal a is this", "lazy the over jumps fox brown quick the dog"]
    ... })
    >>> score(solution, submission, 'id', model_path=model_path, clear_mem=True) > 0
    True
    """
    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)
    invalid_mask = sol_counts != sub_counts
    if invalid_mask.any():
        raise ParticipantVisibleError(
            'At least one submitted string is not a valid permutation of the solution string.'
        )

    # Calculate perplexity for the submitted strings
    sub_strings = [
        ' '.join(s.split()) for s in submission['text'].tolist()
    ]  # Split and rejoin to normalize whitespace
    scorer = PerplexityCalculator(
        model_path=model_path,
        load_in_8bit=load_in_8bit,
    )  # Initialize the perplexity calculator with a pre-trained model
    perplexities = scorer.get_perplexity(
        sub_strings
    )  # Calculate perplexity for each submitted string

    if clear_mem:
        # Just move on if it fails. Not essential if we have the score.
        try:
            scorer.clear_gpu_memory()
        except:
            print('GPU memory clearing failed.')

    return float(np.mean(perplexities))


class PerplexityCalculator:
    """
    Calculates perplexity of text using a pre-trained language model.

    Adapted from https://github.com/asahi417/lmppl/blob/main/lmppl/ppl_recurrent_lm.py

    Parameters
    ----------
    model_path : str
        Path to the pre-trained language model

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    device_map : str, default="auto"
        Device mapping for the model.
    """

    def __init__(
        self,
        model_path: str,
        load_in_8bit: bool = False,
        device_map: str = 'auto',
    ):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
        # Configure model loading based on quantization setting and device availability
        if load_in_8bit:
            if DEVICE.type != 'cuda':
                raise ValueError('8-bit quantization requires CUDA device')
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                device_map=device_map,
            )
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if DEVICE.type == 'cuda' else torch.float32,
                device_map=device_map,
            )

        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')

        self.model.eval()

    def get_perplexity(
        self, input_texts: Union[str, List[str]], debug=False
    ) -> Union[float, List[float]]:
        """
        Calculates the perplexity of given texts.

        Parameters
        ----------
        input_texts : str or list of str
            A single string or a list of strings.

        batch_size : int, default=None
            Batch size for processing. Defaults to the number of input texts.

        debug : bool, default=False
            Print debugging information.

        Returns
        -------
        float or list of float
            A single perplexity value if input is a single string,
            or a list of perplexity values if input is a list of strings.

        Examples
        --------
        >>> import pandas as pd
        >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
        >>> scorer = PerplexityCalculator(model_path=model_path)

        >>> submission = pd.DataFrame({
        ...     'id': [0, 1, 2],
        ...     'text': ["this is a normal english sentence", "thsi is a slihgtly misspelled zr4g sentense", "the quick brown fox jumps over the lazy dog"]
        ... })
        >>> perplexities = scorer.get_perplexity(submission["text"].tolist())
        >>> perplexities[0] < perplexities[1]
        True
        >>> perplexities[2] < perplexities[0]
        True

        >>> perplexities = scorer.get_perplexity(["this is a sentence", "another sentence"])
        >>> all(p > 0 for p in perplexities)
        True

        >>> scorer.clear_gpu_memory()
        """
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []
        with torch.no_grad():
            # Process each sequence independently
            for text in input_texts:
                # Explicitly add sequence boundary tokens to the text
                text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"

                # Tokenize
                model_inputs = self.tokenizer(
                    text_with_special,
                    return_tensors='pt',
                    add_special_tokens=False,
                )

                if 'token_type_ids' in model_inputs:
                    model_inputs.pop('token_type_ids')

                model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                # Get model output
                output = self.model(**model_inputs, use_cache=False)
                logits = output['logits']

                # Shift logits and labels for calculating loss
                shift_logits = logits[..., :-1, :].contiguous()  # Drop last prediction
                shift_labels = model_inputs['input_ids'][..., 1:].contiguous()  # Drop first input

                # Calculate token-wise loss
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                # Calculate average loss
                sequence_loss = loss.sum() / len(loss)
                loss_list.append(sequence_loss.cpu().item())

                # Debug output
                if debug:
                    print(f"\nProcessing: '{text}'")
                    print(f"With special tokens: '{text_with_special}'")
                    print(f"Input tokens: {model_inputs['input_ids'][0].tolist()}")
                    print(f"Target tokens: {shift_labels[0].tolist()}")
                    print(f"Input decoded: {self.tokenizer.decode(model_inputs['input_ids'][0])}")
                    print(f"Target decoded: {self.tokenizer.decode(shift_labels[0])}")
                    print(f"Individual losses: {loss.tolist()}")
                    print(f"Average loss: {sequence_loss.item():.4f}")

        ppl = [exp(i) for i in loss_list]

        if debug:
            print("\nFinal perplexities:")
            for text, perp in zip(input_texts, ppl):
                print(f"Text: '{text}'")
                print(f"Perplexity: {perp:.2f}")

        return ppl[0] if single_input else ppl

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        # Delete model and tokenizer if they exist
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'tokenizer'):
            del self.tokenizer

        # Run garbage collection
        gc.collect()

        # Clear CUDA cache and reset memory stats
        with DEVICE:
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()

In [4]:
sample_submission = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')

In [5]:
# initialize scorer
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
class BeamSearch():
    def __init__(self, beam_width):
        self.beam_width = beam_width

    def _make_candidates_adding_word(self, intermediate_solution, words):
        '''Make candidate text so that word duplication does not occur'''
        candidates = []
        not_used_words = words.copy()
        for used_word in intermediate_solution.split():
            not_used_words.remove(used_word)
        
        candidates += [(intermediate_solution + ' ' + word).lstrip() for word in not_used_words]
        candidates += [(word + ' ' + intermediate_solution).rstrip() for word in not_used_words]

        candidates = list(set(candidates)) # drop duplicate
        
        return candidates
    
    def solve(self, text):
        words = text.split()
        intermediate_solutions = ['']
        eval_cnt = 0

        while len(intermediate_solutions[0].split()) < len(words):
            candidates = []
            for intermediate_solution in intermediate_solutions:
                candidates += self._make_candidates_adding_word(intermediate_solution, words)
            scores = scorer.get_perplexity(candidates)
            eval_cnt += len(scores)
            top_idx = sorted(range(len(scores)), key=lambda i: scores[i])[:self.beam_width]
            intermediate_solutions = [candidates[i] for i in top_idx]

        best_score = min(scores)
        best_solution = candidates[np.argmin(scores)]

        return best_score, best_solution, eval_cnt

In [7]:
params = {
    'beam_width': 3,
}
optimizer = BeamSearch(**params)

In [10]:
sample_submission.loc[0]["text"]

'advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge'

In [12]:
import time
from tqdm import tqdm
scores = []
solutions = []
eval_cnts = []
# for text in tqdm(sample_submission['text']):
text = sample_submission.loc[0]["text"]
score, solution, eval_cnt = optimizer.solve(text)
scores.append(score)
solutions.append(solution)
eval_cnts.append(eval_cnt)

print(scores)
print(solutions)
print(eval_cnts)


[1050.510221115274]
['ornament scrooge mistletoe reindeer elf gingerbread chimney fireplace advent family']
[145]


In [27]:
best_score = 1 << 60
best_blind = -1
for i in range(1000):
    score = scorer.get_perplexity(" "*i + "from and and as and have the in is it of not that the to we with you advent card angel bake beard believe bow candy candle carol cheer cheer chocolate chimney cookie decorations doll dream drive eat eggnog family fireplace fireplace chimney fruitcake game give gifts gingerbread greeting grinch holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night elf nutcracker ornament ornament of the wrapping paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath yuletide")
    if best_score > score:
        best_score = score
        best_blind = i
    print(f"blind, score:{i}, {score}")

blind, score:0, 34.63697922255266
blind, score:1, 40.022953418667065
blind, score:2, 43.444504934395866
blind, score:3, 38.71588573814723
blind, score:4, 37.01518097066168
blind, score:5, 39.634006510082365
blind, score:6, 38.33964103892552
blind, score:7, 40.022953418667065
blind, score:8, 38.71588573814723
blind, score:9, 41.29341833835342
blind, score:10, 40.17959882934157
blind, score:11, 40.73270164287187
blind, score:12, 39.40245503911115
blind, score:13, 40.81233544283598
blind, score:14, 40.81233544283598
blind, score:15, 40.73270164287187
blind, score:16, 39.86691871057298
blind, score:17, 41.2128458403547
blind, score:18, 41.45503620813183
blind, score:19, 41.45503620813183
blind, score:20, 41.132430556936306
blind, score:21, 42.27266353442275
blind, score:22, 41.861853726676614
blind, score:23, 41.943695056893915
blind, score:24, 41.29341833835342
blind, score:25, 42.190180314697045
blind, score:26, 42.190180314697045
blind, score:27, 41.861853726676614
blind, score:28, 41.8

In [28]:
print(best_blind, best_score)

994 20.44172438427552


# 3-3. Consideration 考察
Increasing the beam width will increase the search area and is expected to improve scores. You may want to experiment with different beam widths for different word counts.<br>
Another method, as with the greedy method, is to increase or change the pattern of words to be placed.
In the example implementation, an example of placement on the left is shown in the commented-out section.<br>

ビーム幅を大きくすると、さらに探索範囲が広がりスコアの改善が期待されます。単語数に合わせてビーム幅を変えて実験してみるのも良いでしょう。<br>
また、貪欲法の時と同じく、配置する単語の位置パターンを増やしたり変えてみる方法もあります。実装例では左に配置する例をコメントアウトで記載しています。

# 3-4. Submission 提出
Finally, let's submit the solutions.<br>
最後に、求めた解を提出してみましょう。